# CA2 Clustering resign
By : Joel Poah  
Class :  DAAA/FT/2A/02  
admin no : P2112729

## Goal is to cluster those employees that will resign 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
cluster = pd.read_csv('Company_Employee.csv')
cluster

In [ ]:
cluster.info()

In [ ]:
cluster.isna().any()

# EDA process and finding out more of data
## Fortunately there is no null values

In [ ]:
#importing all modules used 
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import statsmodels.api as sm
from statsmodels.formula.api import ols
import matplotlib.pyplot as plt


## I will split the dataframe to numeric and categorical for now 

In [ ]:
# get columns 0 ,4,5,6,8,9,10,11
num0 = cluster.iloc[:,0]
num1 = cluster.iloc[:,4:7]
num2= cluster.iloc[:,8:12]
num =pd.concat([num0,num1,num2],axis=1)
num

In [ ]:
cat = cluster.iloc [:,1:4]
cat1= cluster.iloc [:,7]
cat2= cluster.iloc[:,12]
cat=pd.concat([cat,cat1,cat2],axis=1)
cat


In [ ]:
#scaling the numerical data
#scale the 0th 1st and 4th column with standard scaler
scaler = StandardScaler()
columns_to_scale1,columns_to_scale2,columns_to_scale3 = num.iloc[:,0:2],num.iloc[:,4],num.iloc[:,7]
columns_to_scale = pd.concat([columns_to_scale1,columns_to_scale2,columns_to_scale3],axis=1)
columns_to_scale = scaler.fit_transform(columns_to_scale)
columns_to_scale = pd.DataFrame(columns_to_scale,columns=['Age','Distance_Company_Home_KM','Salary','Length_of_Service_Years'])
num_scaled=num.drop(['Age','Distance Between Company and Home (KM)','Salary ($)','Length of Service (Years)'],axis=1)

# add columns_to_scale to num_scaled dataframe
num_scaled = pd.concat([num_scaled,columns_to_scale],axis=1)
num_scaled.rename(columns={"Education (1 is lowest, 5 is highest)": "Education","Job Satisfaction (1 is lowest, 4 is highest)":"Job_Satis","Performance Rating (1 is lowest, 4 is highest)":"PerformRating","Work Life Balance (1 is worst, 4 is best)":"WorkLifeBalance"},inplace=True)
num_scaled


## Based On Dendrogram there are 3 huge clusters out of the whole numerical dataframe

In [ ]:
# Perform the necessary imports
from scipy.cluster.hierarchy import linkage, dendrogram

fig = plt.gcf()
# Calculate the linkage: mergings
mergings = linkage(num,method='ward')


# Plot the dendrogram
dendrogram(mergings,
           leaf_rotation=90,
           leaf_font_size=6,
           p = 6
)


fig.set_size_inches(18.5, 10.5)
plt.show()


## Dimensionality curse , there are too many dimesnions in this dataset and would need to be reduced or else model will be overfitted clusters will become less obvious as everything seems to be very close together
### Chi Square test to find independence on categorical data.  
One hot encode will be used for Business Travel,Department,Marital Status 
0 and 1 will be used for Resign Status and Gender

In [ ]:
print("Business travel:",cat.BusinessTravel.unique())
print("Gender :",cat.Gender.unique())
print("Department:",cat['Job Function'].unique())
print("Marital Status:",cat.MaritalStatus.unique())
print("Resign Status:",cat['Resign Status'].unique())
dummy = pd.get_dummies(cat)
dummy.drop('Resign Status_No',axis=1,inplace=True)
# rename num
num.rename(columns={"Education (1 is lowest, 5 is highest)": "Education","Job Satisfaction (1 is lowest, 4 is highest)":"Job_Satis","Performance Rating (1 is lowest, 4 is highest)":"PerformRating","Work Life Balance (1 is worst, 4 is best)":"WorkLifeBalance"},inplace=True)
df_noscale = pd.concat([num[['Education','Job_Satis','PerformRating','WorkLifeBalance']],dummy],axis=1)
df_noscale

In [ ]:
raw = pd.concat([num,dummy],axis=1)
# Clustering silhouette_score
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans
# trying different number of clusters
for k in range(2, 11):
    model = KMeans(n_clusters=k).fit(raw)
    label = model.labels_
    sil_coeff = silhouette_score(raw, label, metric='euclidean')
    print("For n_clusters={}, The Silhouette Coefficient is {:.3f}".format(k,
    sil_coeff))


In [ ]:
from sklearn import datasets
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectKBest
X = df_noscale.drop(['Resign Status_Yes'],axis =1)
# X= pd.concat([num,X],axis=1)
y=df_noscale['Resign Status_Yes'] 

chi_scores = chi2(X,y)
chi_scores
# first array represents chi square values
# second array represents p-values

#null hypothesis attribute is independent
#alternate hypothesis is attribute is not independent

#element 1 and element 2 of p-values is high , it is independent and does not contribute 
#much to our model.

#### null hypothesis attribute is independent  
alternate hypothesis is attribute is not independent

#### If p-values is high , it is independent and does not contribute much to our model.

In [ ]:
p_values = pd.Series(chi_scores[1],index = X.columns)
p_values.sort_values(ascending = False , inplace = True)
p_values.plot.barh(figsize = (20,10))

### Anova test for both categorical and numerical data
If p-values are less than .05, this means that this factors have a statistically significant effect on resignation.
Those marked red should be dropped since p-value is above 0.05 and is not statistically significant.

In [ ]:
df_noscale

In [ ]:
#performing 2 way Anova 
df_noscale.rename(columns={"Resign Status_Yes": "ResignStatus","BusinessTravel_Non-Travel":"BusinessNonTravel","BusinessTravel_Travel_Frequently":"BusinessFrequentlyTravel","Job Function_Human Resources":"Job_HR","Job Function_Research & Development": "Job_RnD","Job Function_Sales":"Job_Sales"},inplace=True)  #renaming the column name
df = pd.concat([num_scaled,df_noscale],axis=1)
model = ols('ResignStatus ~ Distance_Company_Home_KM + MaritalStatus_Married + MaritalStatus_Single + MaritalStatus_Divorced + Education + Job_Satis + PerformRating + WorkLifeBalance + Gender_Female + Gender_Male + BusinessNonTravel + BusinessFrequentlyTravel + Job_HR + Job_RnD + Job_Sales + Age + Length_of_Service_Years + Salary' ,data=df).fit()
anova=sm.stats.anova_lm(model, typ=1).sort_values('PR(>F)',ascending=False).round(10).style.bar(subset=["PR(>F)"],color ='red')
anova.bar(subset=["F"],color ='green')

In [ ]:
def forward_selection(data, target, significance_level=0.05):
    initial_features = data.columns.tolist()
    best_features = []
    while (len(initial_features)>0):
        remaining_features = list(set(initial_features)-set(best_features))
        new_pval = pd.Series(index=remaining_features)
        for new_column in remaining_features:
            model = sm.OLS(target, sm.add_constant(data[best_features+[new_column]])).fit()
            new_pval[new_column] = model.pvalues[new_column]
        min_p_value = new_pval.min()
        if(min_p_value<significance_level):
            best_features.append(new_pval.idxmin())
        else:
            break
    return best_features
df_X = df_noscale.drop(['ResignStatus'],axis =1)
df_y = df_noscale['ResignStatus']
forward_selection(df_X,df_y)

In [ ]:
#importing the necessary libraries
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.linear_model import LinearRegression
# Sequential Forward Selection(sfs)
sfs = SFS(LinearRegression(),
          k_features=11,
          forward=True,
          floating=False,
          scoring = 'r2',
          cv = 0)
sfs.fit(df_X, df_y)
print(sfs.k_feature_names_)  #best features




In [ ]:
sfs1 = SFS(LinearRegression(),
         k_features=(3,11),
         forward=True,
         floating=False,
         cv=0)
sfs1.fit(X, y)
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs
import matplotlib.pyplot as plt
fig1 = plot_sfs(sfs1.get_metric_dict(), kind='std_dev')
plt.title('Sequential Forward Selection (w. StdErr)')
plt.grid()
plt.show()

In [ ]:
def stepwise_selection(data, target,SL_in=0.05,SL_out = 0.05):
    initial_features = data.columns.tolist()
    best_features = []
    while (len(initial_features)>0):
        remaining_features = list(set(initial_features)-set(best_features))
        new_pval = pd.Series(index=remaining_features)
        for new_column in remaining_features:
            model = sm.OLS(target, sm.add_constant(data[best_features+[new_column]])).fit()
            new_pval[new_column] = model.pvalues[new_column]
        min_p_value = new_pval.min()
        if(min_p_value<SL_in):
            best_features.append(new_pval.idxmin())
            while(len(best_features)>0):
                best_features_with_constant = sm.add_constant(data[best_features])
                p_values = sm.OLS(target, best_features_with_constant).fit().pvalues[1:]
                max_p_value = p_values.max()
                if(max_p_value >= SL_out):
                    excluded_feature = p_values.idxmax()
                    best_features.remove(excluded_feature)
                else:
                    break 
        else:
            break
    return best_features
stepwise_selection(df_X,df_y)

### Exhaustive Feature Selection

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from mlxtend.feature_selection import ExhaustiveFeatureSelector as EFS
X = df.drop(['ResignStatus'],axis=1)
y= df['ResignStatus']
knn = KNeighborsClassifier(n_neighbors=1)


efs1 = EFS(knn, 
           min_features=1,
           max_features=6,
           scoring='accuracy',
           print_progress=True) ### Use cross-validation generator here

efs1 = efs1.fit(X, y)

print('Best accuracy score: %.2f' % efs1.best_score_)
print('Best subset (corresponding names):', efs1.best_feature_names_)

In [ ]:
#save efs1 to a txt file
efs1.best_feature_names_
# efs1.best_feature_names_.tolist()
f = open("efs1.txt", "a")
f.write(str(efs1.best_feature_names_))
f.close()

In [ ]:
metric_dict = efs1.get_metric_dict()

fig = plt.figure()
k_feat = sorted(metric_dict.keys())
avg = [metric_dict[k]['avg_score'] for k in k_feat]

upper, lower = [], []
for k in k_feat:
    upper.append(metric_dict[k]['avg_score'] +
                 metric_dict[k]['std_dev'])
    lower.append(metric_dict[k]['avg_score'] -
                 metric_dict[k]['std_dev'])

plt.rcParams["figure.figsize"] = (8,5.5)
plt.fill_between(k_feat,
                 upper,
                 lower,
                 alpha=0.2,
                 color='blue',
                 lw=1)

plt.plot(k_feat, avg, color='blue', marker='o')
plt.ylabel('Accuracy +/- Standard Deviation')
plt.xlabel('Number of Features')
feature_min = len(metric_dict[k_feat[0]]['feature_idx'])
feature_max = len(metric_dict[k_feat[-1]]['feature_idx'])
plt.xticks(k_feat, 
           [str(metric_dict[k]['feature_names']) for k in k_feat], 
           rotation=90)
plt.show()

In [ ]:
# K-means Clustering
from sklearn.datasets import load_iris
from sklearn.cluster import KMeans
import pandas as pd
# fit a Kmeans model to the data
model = KMeans(n_clusters=2)
model.fit(df)
# Visualize the results
import matplotlib.pyplot as plt
y_kmeans = model.predict(df)
# scatter plot the sepal length versus sepal width
plt.scatter(df['Distance_Company_Home_KM'], df['ResignStatus'], c=y_kmeans, s=50, cmap='viridis')
# plot centroids for each cluster
centers = model.cluster_centers_
plt.scatter(centers[:, 0], centers[:, 1], c='red', s=200, alpha=0.8)
plt.xlabel('Salary ($)')
plt.ylabel('age')
plt.show()


In [ ]:
# Clustering silhouette_score
from sklearn.metrics import silhouette_score
from sklearn.datasets import load_iris
from sklearn.cluster import KMeans
# trying different number of clusters
for k in range(2, 11):
    model = KMeans(n_clusters=k).fit(df)
    label = model.labels_
    sil_coeff = silhouette_score(X, label, metric='euclidean')
    print("For n_clusters={}, The Silhouette Coefficient is {:.3f}".format(k,
    sil_coeff))
